In [1]:
import pandas as pd

In [9]:
import requests
import gmaps

from config import g_key

In [ ]:
city_data_df = pd.read_csv ("./data/WeatherPy_Database.csv")
city_data_df

In [15]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [26]:
# enter temp
min_temp = float(input("What is the minimum temperature you would like for your vacation"))
max_temp = float(input("What is the maximum temperature you would like for your vacation"))

What is the minimum temperature you would like for your vacation10
What is the maximum temperature you would like for your vacation200


In [27]:
# Ask the customer if you want it to be raining
rain_amount = input("Do you want it to be raining? (yes/no)")

Do you want it to be raining? (yes/no)no


In [28]:
snow_amount = input("Do you want it to be snowing (yes/no)")

Do you want it to be snowing (yes/no)no


In [29]:
if rain_amount == "no" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] == 0)]
elif rain_amount == "no" and snow_amount == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_amount == "yes" and snow_amount == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0) &
                                          (city_data_df["Snow (inches)"] == 0.0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0) &
                                          (city_data_df["Snow (inches)"] > 0.0)]

In [33]:
filtered_cities_df.head()

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lon,Max Temp,Rain (inches),Snow (inches),Wind Speed
0,0,mataura,87,NZ,overcast clouds,75,-46.19,168.86,47.10,0.0,0.0,1.12
1,1,aksu,69,CN,broken clouds,28,41.12,80.26,71.76,0.0,0.0,3.29
2,2,ushuaia,75,AR,shower rain,86,-54.80,-68.30,37.40,0.0,0.0,4.70
3,3,albany,100,US,overcast clouds,70,42.60,-73.97,81.00,0.0,0.0,5.50
4,4,puerto ayora,83,EC,broken clouds,92,-0.74,-90.35,71.01,0.0,0.0,5.01


In [34]:
filtered_cities_df.count()

 City_ID               523
City                   522
Cloudiness             523
Country                521
Current Description    523
Humidity               523
Lat                    523
Lon                    523
Max Temp               523
Rain (inches)          523
Snow (inches)          523
Wind Speed             523
dtype: int64

In [35]:
filtered_cities_df = filtered_cities_df.dropna()
filtered_cities_df

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lon,Max Temp,Rain (inches),Snow (inches),Wind Speed
0,0,mataura,87,NZ,overcast clouds,75,-46.19,168.86,47.10,0.0,0.0,1.12
1,1,aksu,69,CN,broken clouds,28,41.12,80.26,71.76,0.0,0.0,3.29
2,2,ushuaia,75,AR,shower rain,86,-54.80,-68.30,37.40,0.0,0.0,4.70
3,3,albany,100,US,overcast clouds,70,42.60,-73.97,81.00,0.0,0.0,5.50
4,4,puerto ayora,83,EC,broken clouds,92,-0.74,-90.35,71.01,0.0,0.0,5.01
...,...,...,...,...,...,...,...,...,...,...,...,...
575,575,severobaykalsk,100,RU,overcast clouds,51,55.65,109.32,57.56,0.0,0.0,3.15
576,576,roccastrada,20,IT,few clouds,82,43.01,11.17,60.80,0.0,0.0,5.82
577,577,gizo,22,SB,few clouds,70,-8.10,156.84,84.54,0.0,0.0,3.18
578,578,purranque,100,CL,light intensity shower rain,93,-40.92,-73.17,48.20,0.0,0.0,9.17


In [36]:
filtered_cities_df.count()

 City_ID               520
City                   520
Cloudiness             520
Country                520
Current Description    520
Humidity               520
Lat                    520
Lon                    520
Max Temp               520
Rain (inches)          520
Snow (inches)          520
Wind Speed             520
dtype: int64

In [56]:
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Lat", "Lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lon,Hotel Name
0,mataura,NZ,47.10,-46.19,168.86,
1,aksu,CN,71.76,41.12,80.26,
2,ushuaia,AR,37.40,-54.80,-68.30,
3,albany,US,81.00,42.60,-73.97,
4,puerto ayora,EC,71.01,-0.74,-90.35,


In [62]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/p1ace/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not found... skipping...")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)